In [1]:
import open3d as o3d
import matplotlib.pyplot as plt
import cv2
import numpy as np
import torch
import os
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from open3d.visualization import rendering
# Util function for loading point clouds|
from proj_pcd2img import project_3d_to_2d
from segment3dwithAnno import segment3D_with2DAnno, segment3D_With2DMask, create_2d_mask

# Utils for mesh reconstruction
from alphashape import alphashape
import trimesh
import pymeshfix
from copy import deepcopy


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def paint_color(pcd, color:str):
    if color =="red":
        color_code = [1,0,0]
    elif color =="green":
        color_code = [0,1,0]
    elif color =="blue":
        color_code = [0,0,1]
    elif color == "grey":
        color_code = [0.9, 0.9, 0.9]
    else:
        color_code = [0,0,0]
    return pcd.paint_uniform_color(np.array(color_code))

def create_two_color_mask(rgb_image):
    """
    Create a binary mask for an RGB image with exactly two colors: black and another uniform color.
    
    Args:
        rgb_image: (H, W, 3) RGB image with only two colors (black + one uniform color).
    
    Returns:
        mask: (H, W) binary mask (True for uniform color, False for black).
    """
    # Find the non-black color (assuming it's uniform)
    non_black_pixels = rgb_image[np.any(rgb_image != [255, 255, 255], axis=-1)]
    if len(non_black_pixels) == 0:
        return np.zeros_like(rgb_image[:, :, 0], dtype=bool)  # All black
    
    # Get the unique non-black color (should be just one)
    uniform_color = non_black_pixels[0]
    
    # Create mask: True where pixel matches the uniform color
    mask = np.all(rgb_image == uniform_color, axis=-1)
    return mask

def o3dpcd2img(pcd, width, height, return_camera=False):
    vis = o3d.visualization.Visualizer()
    vis.create_window(width=width, height=height, visible=False)
    vis.get_render_option().point_size = 2
    vis.add_geometry(pcd)
    view_ctl = vis.get_view_control()
    view_ctl.set_zoom(0.6)
    view_ctl.set_lookat(pcd.get_center())
    view_ctl.set_up((1, 0, 0))  # set the positive direction of the x-axis as the up direction
    view_ctl.set_front((0, 0, 1))  # set the positive direction of the x-axis toward you
    vis.update_renderer()
    img = np.array(vis.capture_screen_float_buffer(True))
    depth = np.array(vis.capture_depth_float_buffer(True))
    mask = create_two_color_mask(img)
    if return_camera:
       # https://www.open3d.org/html/python_api/open3d.camera.PinholeCameraIntrinsic.html
       cam = view_ctl.convert_to_pinhole_camera_parameters()
       return img, depth, mask, cam.intrinsic.intrinsic_matrix, cam.extrinsic
    vis.destroy_window()
    return img, depth, mask



def pcd_2_mesh(pcd):
    alpha = 0.5
    mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)
    mesh.compute_vertex_normals()
    return mesh
def rasterize_with_zbuffer(points_3d, H,W):
    """
    Rasterizes 3D points with z-buffering (keeps closest points).
    
    Args:
        points_3d: (N, 3) tensor of 3D points (x, y, z)
        image_size: (height, width) of output image
    
    Returns:
        image: (H, W) binary mask (1 if pixel has a point, else 0)
        depth_buffer: (H, W) depth map (smallest z per pixel)
    """

    device = points_3d.device
    
    # Normalize x, y to [0, 1], keep z for depth
    xy_min = points_3d[:, :2].min(dim=0)[0]
    xy_max = points_3d[:, :2].max(dim=0)[0]
    xy_normalized = (points_3d[:, :2] - xy_min) / (xy_max - xy_min + 1e-6)
    
    # Scale to image dimensions
    u = (xy_normalized[:, 0] * (W - 1)).long()  # [0, W-1]
    v = (xy_normalized[:, 1] * (H - 1)).long()  # [0, H-1]
    z = points_3d[:, 2]  # Depth values
    
    # Initialize buffers
    image = torch.zeros((H, W), device=device)
    depth_buffer = torch.full((H, W), float('inf'), device=device)
    
    # Z-buffering
    for ui, vi, zi in zip(u, v, z):
        if zi < depth_buffer[vi, ui]:
            depth_buffer[vi, ui] = zi
            image[vi, ui] = 1  # Mark occupied pixel
    
    return image, depth_buffer

import torch

def rasterize_with_priority(points_3d, image_size:tuple, axis='xy', highest = True):
    """
    Rasterizes 3D points with priority (highest/lowest first).
    
    Args:
        points_3d: (N, 3) or (B, N, 3) tensor of 3D points.
        image_size: (H, W) output dimensions.
        axis: 
            - 'xy' (top-down view, Z for depth)
            - 'xz' (front view, Y for depth)
            - 'yz' (side view, X for depth)
        priority:
            - 'highest': Points with largest depth are drawn first.
            - 'lowest': Points with smallest depth are drawn first.
    
    Returns:
        image: (H, W) or (B, H, W) binary mask.
        depth_buffer: (H, W) or (B, H, W) depth map.
    """
    H, W = image_size
    device = points_3d.device
    batched = points_3d.dim() == 3
    points_3d[...,2]*=-1
    # Select axes for projection and depth
    if axis == 'xy':
        proj_coords = points_3d[..., :2]  # (N, 2) or (B, N, 2)
        depth = points_3d[..., 2]         # (N,) or (B, N)
    elif axis == 'xz':
        proj_coords = points_3d[..., [0, 2]]
        depth = points_3d[..., 1]
    elif axis == 'yz':
        proj_coords = points_3d[..., [1, 2]]
        depth = points_3d[..., 0]
    else:
        raise ValueError("axis must be 'xy', 'xz', or 'yz'")

    # Normalize projected coordinates to [0, 1]
    proj_min = proj_coords.amin(dim=-2, keepdim=True)  # (1, 2) or (B, 1, 2)
    proj_max = proj_coords.amax(dim=-2, keepdim=True)
    proj_normalized = (proj_coords - proj_min) / (proj_max - proj_min + 1e-6)

    # Scale to image dimensions
    u = (proj_normalized[..., 0] * (W - 1)).long()  # (N,) or (B, N)
    v = (proj_normalized[..., 1] * (H - 1)).long()

    # Sort points by depth (highest or lowest first)
    if highest:
        sort_idx = torch.argsort(depth, descending=True)  # Draw farthest first
    else:
        sort_idx = torch.argsort(depth, descending=False) # Draw closest first

    if batched:
        # Batched sorting (B, N)
        u = torch.gather(u, 1, sort_idx)
        v = torch.gather(v, 1, sort_idx)
        depth = torch.gather(depth, 1, sort_idx)
    else:
        # Single point cloud sorting
        u = u[sort_idx]
        v = v[sort_idx]
        depth = depth[sort_idx]

    # Initialize output
    if batched:
        B = points_3d.shape[0]
        image = torch.zeros((B, H, W), device=device)
        depth_buffer = torch.full((B, H, W), float('inf'), device=device)
    else:
        image = torch.zeros((H, W), device=device)
        depth_buffer = torch.full((H, W), float('inf'), device=device)

    # Rasterize in sorted order
    if batched:
        for b in range(B):
            for ui, vi, zi in zip(u[b], v[b], depth[b]):
                if zi < depth_buffer[b, vi, ui]:  # Overwrite if closer (or farther)
                    depth_buffer[b, vi, ui] = zi
                    image[b, vi, ui] = 1
    else:
        for ui, vi, zi in zip(u, v, depth):
            if zi < depth_buffer[vi, ui]:
                depth_buffer[vi, ui] = zi
                image[vi, ui] = 1

    return image, depth_buffer

In [3]:
pcd = o3d.io.read_point_cloud("multi_tree.ply")
# pcd_color = paint_color(pcd, "grey")

width, height = (640, 480)
img, depth, mask_2d = o3dpcd2img(pcd, width=width, height=height)
img, depth, mask_2d, intrins, extrinsic = o3dpcd2img(pcd, width=width, height=height, return_camera=True)


pcd_selected, mask_2d = segment3D_With2DMask(pcd, mask_2d, intrins, extrinsic)



# print(np.array(pcd_selected.points).shape)
# plt.imshow(mask_2d.detach().cpu().numpy())
# plt.show()
# pcd_selected = segment3D_with2DAnno(pcd_color, segment2d_anno, 480, 640, intrins, extrinsic)
o3d.visualization.draw_geometries([pcd])



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

def rasterize_3dto2D_numpyv3(
    pointcloud, 
    mask_2d=None, 
    img_shape=None,
    min_xyz=None,  
    max_xyz=None,  
    axis='z', 
    highest_first=True,  # Keep highest points in rasterization
    depth_weighting=True,
):
    """
    Rasterizes a point cloud into a 2D image, keeping only the highest points per pixel,
    and filters the point cloud based on the provided mask.
    
    Returns:
        filtered_pointcloud: Points inside the mask (not necessarily highest per pixel).
        raster_image: 2D image with only highest points per pixel.
        raster_filtered_img: Same as raster_image (for compatibility).
    """
    # --- Input checks ---
    if mask_2d is None and img_shape is None:
        raise ValueError("mask_2d or img_shape must be provided.")
    H, W = mask_2d.shape if mask_2d is not None else img_shape
    if mask_2d is None:
        mask_2d = np.ones((H, W), dtype=bool)

    # --- Extract depth ---
    depth = pointcloud[:, {'x': 0, 'y': 1, 'z': 2}[axis]]
    norm_depth = (depth - depth.min()) / (depth.max() - depth.min() + 1e-6)
    if highest_first:
        norm_depth = 1.0 - norm_depth  # Highest depth = 1 (red)

    # --- Custom colormap (Red=high, Blue=mid, Green=low) ---
    cmap = plt.get_cmap('gist_rainbow')

    # --- Project to 2D ---
    if axis == 'z':
        coords = pointcloud[:, :2]  # XY projection
    elif axis == 'y':
        coords = pointcloud[:, [0, 2]]  # XZ projection
        coords[:, 1] *= -1
    elif axis == 'x':
        coords = pointcloud[:, [1, 2]]  # YZ projection
        coords[:, 1] *= -1

    # Normalize coordinates to [0, 1]
    min_coord = np.array([min_xyz[0], min_xyz[1]]) if min_xyz is not None else coords.min(axis=0)
    max_coord = np.array([max_xyz[0], max_xyz[1]]) if max_xyz is not None else coords.max(axis=0)
    coords_normalized = (coords - min_coord) / (max_coord - min_coord + 1e-6)
    
    # Convert to pixel indices
    u = (coords_normalized[:, 0] * (W - 1)).astype(int)
    v = (coords_normalized[:, 1] * (H - 1)).astype(int)

    # --- Filter points based on mask ---
    valid_mask = (u >= 0) & (u < W) & (v >= 0) & (v < H) & mask_2d[v, u]
    filtered_pointcloud = pointcloud[valid_mask]
    u_valid, v_valid = u[valid_mask], v[valid_mask]
    norm_depth_valid = norm_depth[valid_mask]

    # --- Rasterize: Keep only the highest point in each pixel ---
    raster_image = np.zeros((H, W, 3), dtype=np.float32)
    depth_buffer = np.full((H, W), -np.inf)  # Track highest depth per pixel

    # Sort points by depth (highest first)
    sorted_order = np.argsort(norm_depth_valid)[::-1]
    u_sorted = u_valid[sorted_order]
    v_sorted = v_valid[sorted_order]
    norm_depth_sorted = norm_depth_valid[sorted_order]

    # Assign colors to the highest point in each pixel
    for i in range(len(u_sorted)):
        u_pix, v_pix = u_sorted[i], v_sorted[i]
        if norm_depth_sorted[i] > depth_buffer[v_pix, u_pix]:
            depth_buffer[v_pix, u_pix] = norm_depth_sorted[i]
            raster_image[v_pix, u_pix] = cmap(norm_depth_sorted[i])[:3]  # RGB only

    return filtered_pointcloud, raster_image, raster_image.copy()

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

def torch_lexsort(a, dim=-1):
    assert dim == -1  # Transpose if you want differently
    assert a.ndim == 2  # Not sure what is numpy behaviour with > 2 dim
    # To be consistent with numpy, we flip the keys (sort by last row first)
    a_unq, inv = torch.unique(a.flip(0), dim=dim, sorted=True, return_inverse=True)
    return torch.argsort(inv)

def rasterize_3dto2D_torch(
    pointcloud, 
    mask_2d: np.ndarray=None, 
    img_shape: tuple=None,
    min_xyz: tuple=None,  # (min_x, min_y, min_z)
    max_xyz: tuple=None,  # (max_x, max_y, max_z)
    axis='z', 
    highest_first=True,
    depth_weighting=True
):
    """
    Rasterize point cloud with explicit bounds and depth-based weighting.
    Higher values → Red
    Mid values → Blue
    Lowest values → Green
    
    Args:
        pointcloud: (N, 3) array of 3D points.
        mask_2d: (H, W) binary mask (True = keep point).
        img_shape: (H, W) if mask_2d is not None.
        min_xyz: Tuple of (min_x, min_y, min_z) bounds.
        max_xyz: Tuple of (max_x, max_y, max_z) bounds.
        axis: 'xy', 'xz', or 'yz' (projection plane).
        highest_first: If True, prioritize farthest points; else closest.
        depth_weighting: If True, farther points have lower values.
    
    Returns:
        filtered_pointcloud: (M, 3) array (M <= N).
        raster_image: (H, W) binary | (H, W, 3) RGB image (Red=High, Blue=Mid, Green=Low).
        raster_filtered_img: (H, W) binary of masked points | (H, W, 3) RGB image of masked points.
    """
    assert not(mask_2d is None and img_shape is None), "mask_2d or img_shape must be present for rasterization"
    device = pointcloud.device
    dtype = pointcloud.dtype
    if mask_2d is not None:
        H, W = mask_2d.shape
        if isinstance(mask_2d,np.ndarray):
            mask_2d = torch.tensor(mask_2d, dtype=torch.bool, device=device)
    else:
        H, W = img_shape
    
    xyz = pointcloud[:,:3]
    if axis == 'z':
        depth = pointcloud[:, 2]  # Z-axis for XY projection
        coords = xyz[:, :2]
        min_coord = torch.tensor([min_xyz[0], min_xyz[1]]) if min_xyz is not None else coords.min(dim=0)[0]
        max_coord = torch.tensor([max_xyz[0], max_xyz[1]]) if max_xyz is not None else coords.max(dim=0)[0]
    elif axis == 'y':
        depth = pointcloud[:, 1]  # Y-axis for XZ projection
        coords[:,1] = coords[:,1] * -1
        min_coord = torch.tensor([min_xyz[0], min_xyz[2]]) if min_xyz is not None else coords.min(dim=0)[0]
        max_coord = torch.tensor([max_xyz[0], max_xyz[2]]) if max_xyz is not None else coords.max(dim=0)[0]
    elif axis == 'x':
        depth = pointcloud[:, 0]  # X-axis for YZ projection
        coords[:,1] = coords[:,1] * -1
        min_coord = torch.tensor([min_xyz[1], min_xyz[2]]) if min_xyz is not None else coords.min(dim=0)[0]
        max_coord = torch.tensor([max_xyz[1], max_xyz[2]]) if max_xyz is not None else coords.max(dim=0)[0]
    else:
        raise ValueError("axis must be 'x', 'y', or 'z'")

    # Normalize to [0, 1] 
    coords_normalized = (coords - min_coord) / (max_coord - min_coord + 1e-6)
    norm_depth = (depth - depth.min()) / (depth.max() - depth.min() + 1e-6)
    if highest_first:
        norm_depth = 1.0 - norm_depth
    
    # Scale to mask dimensions and round to integer indices
    u = (coords_normalized[:, 0] * (W - 1)).long()
    v = (coords_normalized[:, 1] * (H - 1)).long()
    
    # --- Step 3: Filter points using the mask ---
    # Raster_filtered use sorted, 
    valid_within_bounds = (0 <= u) & (u < W) & (0 <= v) & (v < H)
    if mask_2d is None:
        valid_within_bounds_n_mask = valid_within_bounds.nonzero().squeeze(1)
    else:
        valid_within_bounds_n_mask = torch.zeros_like(u, dtype=torch.bool)
        valid_within_bounds_n_mask[valid_within_bounds.nonzero().squeeze(1)] = mask_2d[v[valid_within_bounds],u[valid_within_bounds]]
        valid_within_bounds_n_mask = valid_within_bounds_n_mask.nonzero().squeeze(1)
    
    filtered_pointcloud = pointcloud[valid_within_bounds_n_mask]
    # Apply depth weighting if enabled
    if depth_weighting:
        cmap = plt.get_cmap('gist_rainbow')
        raster_image = torch.zeros((H, W, 3), dtype=dtype, device=device)
        raster_filtered_img = torch.zeros((H, W, 3), dtype=dtype, device=device)
        
        
        # --- Filter points within bounds ---
        u_valid, v_valid = u[valid_within_bounds], v[valid_within_bounds]
        norm_depth_valid = norm_depth[valid_within_bounds]
        
        # --- Filter points within bounds and 2dMask ---
        u_valid_mask, v_valid_mask = u[valid_within_bounds_n_mask], v[valid_within_bounds_n_mask]
        norm_depth_valid_mask = norm_depth[valid_within_bounds_n_mask]
        
        # --- Vectorized occlusion handling within bounds---
        pixel_keys = v_valid * W + u_valid
        # Sort by highest first
        sort_order = torch_lexsort(torch.stack([pixel_keys, -norm_depth_valid]))
        u_valid, v_valid = u_valid[sort_order], v_valid[sort_order]
        norm_depth_valid = norm_depth_valid[sort_order]
        # Find the first occurence of each pixel        
        unique_indices = torch.unique(v_valid * W + u_valid, return_inverse=False, return_counts=False)
        u_valid, v_valid = u_valid[unique_indices], v_valid[unique_indices]
        raster_image[v_valid, u_valid] = torch.tensor(
                cmap(norm_depth_valid[unique_indices].detach().cpu().numpy())[:, :3], dtype=dtype, device=device
            )
        
        # --- Vectorized occlusion handling within bounds and mask---
        if mask_2d is None:
            raster_filtered_img = raster_image.copy()
        else:
            pixel_keys = v_valid_mask * W + u_valid_mask
            # Sort by highest first
            sort_order = torch_lexsort(torch.stack([pixel_keys, -norm_depth_valid_mask]))
            u_valid_mask, v_valid_mask = u_valid_mask[sort_order], v_valid_mask[sort_order]
            norm_depth_valid_mask = norm_depth_valid_mask[sort_order]
            # Find the first occurence of each pixel
            unique_indices = torch.unique(v_valid_mask * W + u_valid_mask, return_inverse=False, return_counts=False, dim=0)
            u_valid_mask, v_valid_mask = u_valid_mask[unique_indices], v_valid_mask[unique_indices]
            raster_filtered_img[v_valid_mask, u_valid_mask] = torch.tensor(
                cmap(norm_depth_valid_mask[unique_indices].detach().cpu().numpy())[:, :3], dtype=dtype, device=device
            )
    else:
        # Binary version (original behavior)
        raster_image = torch.zeros((H, W), dtype=torch.bool, device=device)
        raster_filtered_img = torch.zeros((H, W), dtype=torch.bool, device=device)
        raster_image[v[valid_within_bounds], u[valid_within_bounds]] = True
        if mask_2d is None:
            raster_filtered_img = raster_image.copy()
        else:
            raster_filtered_img[v[valid_within_bounds_n_mask], u[valid_within_bounds_n_mask]] = True
    
    
    return filtered_pointcloud, raster_image, raster_filtered_img

In [12]:
import numpy as np
import open3d as o3d
from raster_pcd2img import rasterize_3dto2D, rasterize_3dto2D_numpy


# Example usage
if __name__ == "__main__":
    # Generate a sample point cloud (N, 3)
    pointcloud = np.random.rand(1000, 3) * 10
    
    # Create a dummy mask (H, W) (e.g., circle mask)
    H, W = 480, 480
    y, x = np.ogrid[:H, :W]
    center = (W//2, H//2)
    radius = 150
    mask_2d = (x - center[0])**2 + (y - center[1])**2 <= radius**2
    min_pts = np.array(pcd.points).min(axis=0) 
    max_pts = np.array(pcd.points).max(axis=0) 
    # Case 1: Highest-first (farthest points prioritized)
    filtered_pc_highest, raster_img, raster_img_filtered = rasterize_3dto2D(
        np.array(pcd.points), 
        mask_2d, 
        img_shape=(H,W),
        axis='z',
        min_xyz=min_pts,
        max_xyz=max_pts,
        highest_first=False,
        depth_weighting=True
    )
    # plt.imshow(raster_img, cmap="gray")
    # plt.show()
    # plt.imshow(raster_img_filtered*255, cmap="gray")
    # plt.show()
    
    filtered_pc_highest, raster_img, raster_img_filtered = rasterize_3dto2D(
        torch.tensor(np.array(pcd.points)), 
        mask_2d, 
        #img_shape=(H,W),
        axis='y',
        highest_first=False,
        depth_weighting=True
    )
    
    filtered_pc_highest, raster_img, raster_img_filtered = rasterize_3dto2D_torch(
        torch.tensor(np.array(pcd.points)), 
        mask_2d, 
        img_shape=(H,W),
        axis='z',
        highest_first=True,
        depth_weighting=True,
    )
    plt.imshow(raster_img)
    plt.show()
    plt.imshow(raster_img_filtered)
    plt.show()

    
    
    # Visualize
    pcd_original = o3d.geometry.PointCloud()
    pcd_original.points = o3d.utility.Vector3dVector(pointcloud)
    pcd_original.paint_uniform_color([0.5, 0.5, 0.5])  # Gray
    
    pcd_highest = o3d.geometry.PointCloud()
    pcd_highest.points = o3d.utility.Vector3dVector(filtered_pc_highest)
    pcd_highest.paint_uniform_color([1, 0, 0])  # Red (farthest kept)
    
    
    o3d.visualization.draw_geometries([pcd, pcd_highest])

IndexError: index 104489 is out of bounds for dimension 0 with size 26138

In [ ]:
x = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
x_mask = np.array([0, 1, 0, 1, 0, 1, 0, 1, 0, 1], dtype=bool)
x[np.array([4,5])]
x[x_mask]

array([1, 3, 5, 7, 9])

In [ ]:
x=0.02
print((1-x)/x)

49.0
